In [ ]:
import open3d as o3d
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import copy

from ptCloudParsingUtils import *

%load_ext autoreload
%autoreload 2
%autosave 180
    
print(o3d.__version__)

# Open3d 0.18.0 documentation

https://www.open3d.org/docs/release/index.html

In [ ]:
pts_voxspec = np.load('/home/daniel-choate/ASAR/s2/RefMats/voxel_testpts.npy')
homo_trans = np.load('/home/daniel-choate/ASAR/s2/RefMats/homotrans_testpts.npy')
poses = np.load('/home/daniel-choate/ASAR/s2/SfM/poses_ghall.npy')
scene_pts = np.load('/home/daniel-choate/ASAR/s2/RefMats/tuftsww_3dpts.npy')
vox_dist = np.load('/home/daniel-choate/ASAR/s2/TerrainNav/vox_dist_test.npy')

In [ ]:
pcd = o3d.geometry.PointCloud()
points = np.random.rand(1000,3)
pcd.points = o3d.utility.Vector3dVector(points)

o3d.visualization.draw_plotly([pcd])

# Plotting poses

In [ ]:
# Create Open3D visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='3D Plot with Pose Axes and Point Cloud', width=1000, height=1000)

# Add coordinate axes
axes = o3d.geometry.TriangleMesh.create_coordinate_frame()

# loop through poses 
for pose in poses:
    homog_t = pose
    axes1 = copy.deepcopy(axes).transform(homog_t)
    vis.add_geometry(axes1)
    

vis.add_geometry(axes)

# Run the visualizer
vis.run()
vis.destroy_window()

# Plotting ellipses over voxel

In [ ]:
muh_pts, cov_pts = fit_gaussian(pts_voxspec)
vox_ID = 642
color_red = [1,0,0]

In [ ]:
# Create the visualization 
vis = o3d.visualization.Visualizer()
vis.create_window()

# Plotting specific voxel 
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
pts_spec = o3d.geometry.PointCloud()
pts_spec.points = o3d.utility.Vector3dVector(pts_voxspec)
pts_spec.paint_uniform_color([1,0,0])

# Create an initial sphere mesh to be transformed into an ellipse
muh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius = 1, resolution = 20) # 20 is the default
# muh_sphere.translate(muh_pts, relative=True)
muh_sphere.transform(homo_trans)
muh_sphere.paint_uniform_color([0,0,.5])


# Add geometry 
vis.add_geometry(pts_spec)
vis.add_geometry(axes)

# ADD MUH
vis.add_geometry(muh_sphere)


# Color SPECIFIED VOXEL 
# vis = plot_spec_vox(vis, scene_pts, vox_dist, vox_ID, color_red



# Run the visualization
vis.run()
vis.destroy_window()

In [ ]:
# Plot points from voxel with mean 

# Plotting specific voxel 
# First, plot original scene 

vis = o3d.visualization.Visualizer()
vis.create_window()
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
pts_spec = o3d.geometry.PointCloud()
# muh_pts = o3d.geometry.PointCloud()
# muh_pts.paint_uniform_color([0,0,1])

muh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius = 1)
# muh_sphere = o3d.geometry.TriangleMesh.create_tetrahedron()
# muh_sphere.translate(muh_pts, relative=True)
muh_sphere.transform(homo_trans)
muh_sphere.paint_uniform_color([0,0,.5])

# muh_box = muh_sphere.get_minimal_oriented_bounding_box()
muh_box = muh_sphere.get_oriented_bounding_box()
# muh_box.paint_uniform_color([0,0,0])
# Get the 8 corner points of the OBB
points = np.array(muh_box.get_box_points())
# Define the edges of the OBB

edges = [
    [0, 1], [0, 3], [6, 1], [3, 6],  # Bottom face
    [2, 7], [4, 7], [4, 5], [2, 5],  # Top face
    [0, 2], [1, 7], [4, 6], [3, 5]   # Vertical edges
]

# Create a LineSet object
lines = o3d.geometry.LineSet()

# Set points and edges
lines.points = o3d.utility.Vector3dVector(points)
lines.lines = o3d.utility.Vector2iVector(edges)

# Optionally, set the color of the lines to black
# lines.colors = o3d.utility.Vector3dVector([[0, 0, 0]] * len(edges))
lines.paint_uniform_color([0,0,0])



# Add geometry 
vis.add_geometry(pts_spec)
vis.add_geometry(axes)


# ADD MUH
vis.add_geometry(muh_sphere)
vis.add_geometry(muh_box)
vis.add_geometry(lines)


# Color SPECIFIED VOXEL 
vis = plot_spec_vox(vis, scene_pts, vox_dist, vox_ID, color_red)

# Run the visualization
vis.run()
vis.destroy_window()

In [ ]:
# Create a function for plotting an ellipse and bounding box on a voxel
def plot_ellipse_box(vis, muh, cov, color):
    """ Plotting an ellipse and bounding box for a group of pts
    in a voxel 
    Inputs: visualization, mean of the points, covariance of the points, color of ellipse
    Outputs: Visualization with ellipse and bounding box
    """
    # Get eigenvalues and eigenvectors from covariance matrix 
    eig_vals, eig_vect = np.linalg.eig(cov)
    d_eig = (np.sqrt(eig_vals))*np.eye(3)
    # Multiply by two for two standard deviations 
    d_eig = d_eig*2

    # Form the homogeneous transformation matrix 
    A_trans = np.matmul(eig_vect, d_eig)
    # Needs to be 4x4 for homogeneous, adding translation 
    homo_trans = np.append(np.append(A_trans, np.zeros([3,1]), axis = 1), np.array([[0,0,0,1]]), axis = 0)
    homo_trans[:3,-1] = muh

    # Create a sphere mesh and transform into matrix using open3d
    muh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius = 1)
    muh_sphere.transform(homo_trans)
    muh_sphere.paint_uniform_color(color)

    # Create a bounding box for the sphere 
    muh_box = muh_sphere.get_oriented_bounding_box()

    # Attain points and lines to plot the bounding box
    points = np.array(muh_box.get_box_points())
    
    # Correlate points to form edges of the box
    # NOTE: DO NOT MESS with this form for vertices
    edges = [
    [0, 1], [0, 3], [6, 1], [3, 6],  # Bottom face
    [2, 7], [4, 7], [4, 5], [2, 5],  # Top face
    [0, 2], [1, 7], [4, 6], [3, 5]   # Vertical edges
    ]

    # Create a LineSet object
    lines = o3d.geometry.LineSet()

    # Set points and edges
    lines.points = o3d.utility.Vector3dVector(points)
    lines.lines = o3d.utility.Vector2iVector(edges)

    # Optionally, set the color of the lines to black
    lines.colors = o3d.utility.Vector3dVector([[0, 0, 0]] * len(edges))

    # Add the resulting geometries
    vis.add_geometry(muh_sphere)
    vis.add_geometry(muh_box)
    vis.add_geometry(lines)

    return vis

# Plot ellipses and boxes using function 

In [ ]:
# Plot points from voxel with mean 

# Plotting specific voxel 
# First, plot original scene 

vis = o3d.visualization.Visualizer()
vis.create_window()

